In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, date, timedelta

In [2]:
#Opening Realtime data with ERA5 files
jan_t_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_t.nc')
feb_t_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_t.nc')
jan_u_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_u.nc')
feb_u_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_u.nc')
jan_v_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_v.nc')
feb_v_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_v.nc')
jan_g_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_g.nc')
feb_g_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_g.nc')
jan_pv_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_pv.nc')
feb_pv_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_pv.nc')

In [3]:
#Regridding ERA5 data using xesmf

ds_out = xr.Dataset(
    {
        "lon": (["lon"], np.arange(0, 360, 1.5)),
        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
    }
)

regridder = xe.Regridder(jan_t_ds, ds_out, "bilinear")
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_401x1440_121x240.nc 
Reuse pre-computed weights? False 
Input grid shape:           (401, 1440) 
Output grid shape:          (121, 240) 
Periodic in longitude?      False

In [4]:
jan_t_rgrd = regridder(jan_t_ds)
feb_t_rgrd = regridder(feb_t_ds)
jan_u_rgrd = regridder(jan_u_ds)
feb_u_rgrd = regridder(feb_u_ds)
jan_v_rgrd = regridder(jan_v_ds)
feb_v_rgrd = regridder(feb_v_ds)
jan_g_rgrd = regridder(jan_g_ds)
feb_g_rgrd = regridder(feb_g_ds)
jan_pv_rgrd = regridder(jan_pv_ds)
feb_pv_rgrd = regridder(feb_pv_ds)

In [5]:
t_ds = xr.concat([jan_t_rgrd, feb_t_rgrd], dim='time')
u_ds = xr.concat([jan_u_rgrd, feb_u_rgrd], dim='time')
v_ds = xr.concat([jan_v_rgrd, feb_v_rgrd], dim='time')
g_ds = xr.concat([jan_g_rgrd, feb_g_rgrd], dim='time')
pv_ds = xr.concat([jan_pv_rgrd, feb_pv_rgrd], dim='time')

In [7]:
#Lat/lons wanted for NH
lat_min = 20
lat_max = 80
lon_max = 300
lon_min = 200

levels_wanted = [1, 10, 50, 100, 250, 500, 700, 850, 1000]

In [8]:
t_rd = t_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted)
u_rd = u_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted)
v_rd = v_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted)
g_rd = g_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted)
pv_rd = pv_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted)

In [9]:
t_rd_0z = t_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted, time=t_ds.time.dt.hour == 0)
u_rd_0z = u_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted, time=u_ds.time.dt.hour == 0)
v_rd_0z = v_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted, time=v_ds.time.dt.hour == 0)
g_rd_0z = g_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted, time=g_ds.time.dt.hour == 0)
pv_rd_0z = pv_ds.sel(lat=slice(lat_min,lat_max), lon=slice(lon_min,lon_max), level=levels_wanted, time=pv_ds.time.dt.hour == 0)

In [10]:
t_rd_ds = t_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/ERA5_rd/t_rd.nc",
                         mode='w')
u_rd_ds = u_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/ERA5_rd/u_rd.nc",
                         mode='w')
v_rd_ds = v_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/ERA5_rd/v_rd.nc",
                         mode='w')
g_rd_ds = g_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/ERA5_rd/g_rd.nc",
                         mode='w')
pv_rd_ds = pv_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/ERA5_rd/pv_rd.nc",
                         mode='w')